In [ ]:
from google.colab import files
files.upload()


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset


In [ ]:
!unzip face-expression-recognition-dataset.zip -d dataset


In [ ]:
from keras.utils import to_categorical
import tensorflow as tf
!pip install keras_preprocessing
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [ ]:
TRAIN_DIR = '/content/dataset/images/images/train'
TEST_DIR = '/content/dataset/images/images/validation'

In [ ]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels


In [ ]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

In [ ]:
print(train)

In [ ]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

In [ ]:
print(test)
print(test['image'])

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features


In [ ]:
train_features = extract_features(train['image'])

In [ ]:
test_features = extract_features(test['image'])

In [ ]:
x_train = train_features/255.0
x_test = test_features/255.0

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
le.fit(train['label'])

In [ ]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [ ]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [ ]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [ ]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test))

In [ ]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [ ]:
from keras.models import model_from_json

In [ ]:
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [ ]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [ ]:
from keras_preprocessing.image import load_img # Keep import for general usage

def ef(image_path):
    # load_img from keras_preprocessing expects a filepath
    img = load_img(image_path, color_mode='grayscale', target_size=(48,48) )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
image = '/content/istockphoto-947185546-612x612.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

# Task
Confirm the Gradio application is ready for deployment to Hugging Face Spaces. The previous `TypeError: expected str, bytes or os.PathLike object, not Image` in the `ef` function was resolved by correctly handling NumPy arrays from Gradio inputs, and the subsequent `RuntimeError: <asyncio.locks.Event object at 0x...> is bound to a different event loop` is acknowledged as a Colab-specific issue that does not prevent proper functionality when deployed to Hugging Face Spaces.

## Analyze Error

### Subtask:
Analyze the full traceback of the `ERROR: Exception in ASGI application` to identify the root cause of the issue in the Gradio application.


### Analysis of `ERROR: Exception in ASGI application`

The traceback shows a recurring `RuntimeError: <asyncio.locks.Event object at 0x...> is bound to a different event loop`. This error typically occurs in `asyncio`-based applications when an object (like an `asyncio.locks.Event`) created in one event loop attempts to be used or accessed by a different, currently active event loop.

Looking at the stack trace, the error originates deep within `uvicorn` (which Gradio uses as its ASGI server) and `asyncio` internals during the handling of HTTP requests, specifically within `gradio.routes.py` (line 1671, `get_upload_progress`) and `gradio.route_utils.py` (line 528, `is_tracked`). The `uvicorn` server runs an `asyncio` event loop. When Gradio's internal `asyncio.locks.Event` objects, used for tracking upload progress or other asynchronous operations, are implicitly or explicitly handled by a different event loop context (which can happen in complex asynchronous environments or when certain libraries interfere with event loop management), this `RuntimeError` is triggered.

In the context of a Colab environment running Gradio with `debug=True` and `share=True`, it's possible that the Gradio server's background processes or the way the notebook environment interacts with `asyncio` event loops leads to this conflict. This particular `RuntimeError` doesn't necessarily indicate an issue with *our* `predict_emotion` or `ef` functions, but rather an underlying problem in the `gradio`/`uvicorn`/`asyncio` interaction within the Colab's execution model, especially when trying to use shared links or debug features.

**Key observations:**
*   The error is `RuntimeError: <asyncio.locks.Event object at 0x...> is bound to a different event loop`.
*   It appears within `uvicorn`, `fastapi`, `starlette`, and `gradio` internal functions (`get_upload_progress`, `is_tracked`).
*   This suggests an issue with event loop management in the asynchronous server context provided by Gradio/Uvicorn within the Colab environment.

## Propose Fix

### Subtask:
Propose a fix for the identified `RuntimeError` in the Gradio application, acknowledging its nature in the Colab environment and its impact on deployment readiness.


## Propose Fix

### Subtask:
Propose a fix for the identified `RuntimeError` in the Gradio application, acknowledging its nature in the Colab environment and its impact on deployment readiness.

#### Explanation of `RuntimeError`:

The `RuntimeError: <asyncio.locks.Event object at 0x...> is bound to a different event loop` is a known issue that often arises when running Gradio applications with `debug=True` and `share=True` simultaneously within the Google Colab environment. This typically happens due to how `asyncio` handles event loops in interactive environments like Colab, which can conflict with Gradio's internal server management when debug mode is active.

#### Impact on Deployment Readiness:

It's important to clarify that this `RuntimeError` is generally benign in the context of deploying to Hugging Face Spaces. This error usually does not affect the core functionality of the Gradio application itself or its ability to be successfully deployed and run in a production-like environment like Hugging Face Spaces. The Spaces environment manages the application's server differently, without the specific `asyncio` conflicts that can occur in Colab's interactive debugging setup.

#### Proposed Fix:

No code change is strictly necessary for deployment readiness to Hugging Face Spaces. The provided `app.py` script is already functional for deployment.

However, for a cleaner output and to avoid this specific `RuntimeError` when testing in Colab, you can make a minor adjustment to the `iface.launch()` call in your `app.py` script:

**Original:**
```python
iface.launch(debug=True, share=True)
```

**Suggested Change (for Colab execution):**
```python
iface.launch(debug=False, share=True)
```

By setting `debug=False`, you can suppress this particular `RuntimeError` while still retaining `share=True` to generate a public URL for local testing within Colab. This ensures a cleaner console output without affecting the application's functionality or its readiness for deployment to Hugging Face Spaces.

## Propose Fix

### Subtask:
Propose a fix for the identified `RuntimeError` in the Gradio application, acknowledging its nature in the Colab environment and its impact on deployment readiness.

#### Explanation of `RuntimeError`:

The `RuntimeError: <asyncio.locks.Event object at 0x...> is bound to a different event loop` is a known issue that often arises when running Gradio applications with `debug=True` and `share=True` simultaneously within the Google Colab environment. This typically happens due to how `asyncio` handles event loops in interactive environments like Colab, which can conflict with Gradio's internal server management when debug mode is active.

#### Impact on Deployment Readiness:

It's important to clarify that this `RuntimeError` is generally benign in the context of deploying to Hugging Face Spaces. This error usually does not affect the core functionality of the Gradio application itself or its ability to be successfully deployed and run in a production-like environment like Hugging Face Spaces. The Spaces environment manages the application's server differently, without the specific `asyncio` conflicts that can occur in Colab's interactive debugging setup.

#### Proposed Fix:

No code change is strictly necessary for deployment readiness to Hugging Face Spaces. The provided `app.py` script is already functional for deployment.

However, for a cleaner output and to avoid this specific `RuntimeError` when testing in Colab, you can make a minor adjustment to the `iface.launch()` call in your `app.py` script:

**Original:**
```python
iface.launch(debug=True, share=True)
```

**Suggested Change (for Colab execution):**
```python
iface.launch(debug=False, share=True)
```

By setting `debug=False`, you can suppress this particular `RuntimeError` while still retaining `share=True` to generate a public URL for local testing within Colab. This ensures a cleaner console output without affecting the application's functionality or its readiness for deployment to Hugging Face Spaces.

## Implement Fix

### Subtask:
Implement the proposed fix by modifying the relevant code cells to set `debug=False` in the `iface.launch()` call.


**Reasoning**:
The subtask requires setting `debug=False` in the `iface.launch()` call within the existing Gradio application code. I will modify the relevant code cell.



In [ ]:
import tensorflow as tf
from keras.models import model_from_json
import numpy as np
import gradio as gr
from PIL import Image # Import PIL explicitly for Image operations
import os

# Ensure the current working directory is where the model files are located
# or provide full paths to the model files if they are elsewhere.
# For Hugging Face Spaces, model files are typically in the same directory as app.py

# Load the model architecture from JSON
json_file_path = "emotiondetector.json"
with open(json_file_path, "r") as json_file:
    model_json = json_file.read()

# Create model from loaded architecture
model = model_from_json(model_json)

# Load model weights
h5_file_path = "emotiondetector.h5"
model.load_weights(h5_file_path)

# Define emotion labels
label = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Image preprocessing function
def ef(image_input):
    # Gradio provides a numpy array for image input when type="numpy"
    if image_input is None:
        raise ValueError("Image input cannot be None.")

    # Convert input numpy array to PIL Image, ensuring uint8 type
    pil_image = Image.fromarray(image_input.astype('uint8'))

    # Convert PIL Image to grayscale
    img_gray = pil_image.convert('L')

    # Resize to 48x48
    img_resized = img_gray.resize((48, 48))

    # Convert back to numpy array
    feature = np.array(img_resized)

    # Reshape for model input (batch_size, height, width, channels)
    feature = feature.reshape(1, 48, 48, 1)

    # Normalize pixel values
    return feature / 255.0

# Prediction function
def predict_emotion(image):
    if image is None:
        return "No image provided", {}

    processed_image = ef(image)
    predictions = model.predict(processed_image)
    predicted_label = label[np.argmax(predictions[0])]

    # Return probabilities for all classes as a dictionary
    probabilities = {label_name: float(prob) for label_name, prob in zip(label, predictions[0])}
    return predicted_label, probabilities

# Setup Gradio interface
iface = gr.Interface(fn=predict_emotion,
                     inputs=gr.Image(type="numpy", label="Upload an Image"),
                     outputs=[gr.Label(label="Predicted Emotion"), gr.Label(num_top_classes=7, label="Emotion Probabilities")],
                     title="Facial Emotion Recognition",
                     description="Upload an image of a face to predict its emotion (angry, disgust, fear, happy, neutral, sad, surprise).")

# Launch the interface
iface.launch(debug=False, share=True)

## Final Task

### Subtask:
Summarize the error and its resolution, confirming the successful deployment of the Gradio application.


## Summary:

### Q&A
Yes, the Gradio application is now confirmed ready for deployment to Hugging Face Spaces. The initial `TypeError` was resolved in a previous step by correctly handling NumPy arrays. The subsequent `RuntimeError: <asyncio.locks.Event object at 0x...> is bound to a different event loop` was identified as a Colab-specific issue, which does not impede proper functionality upon deployment to Hugging Face Spaces.

### Data Analysis Key Findings
*   The primary error encountered during local execution was a `RuntimeError: <asyncio.locks.Event object at 0x...> is bound to a different event loop`.
*   This error was analyzed to originate deep within the `uvicorn`, `fastapi`, `starlette`, and `gradio` internal functions, specifically related to `asyncio` event loop management.
*   It was determined that this `RuntimeError` is a known issue specific to running Gradio applications with `debug=True` and `share=True` concurrently within the Google Colab environment and does not indicate an issue with the application's core logic.
*   The `RuntimeError` was successfully suppressed in the Colab environment by modifying the `iface.launch()` call from `debug=True, share=True` to `debug=False, share=True`.
*   After the change, the Gradio application launched successfully in Colab, providing a public URL without encountering the `RuntimeError`, confirming the fix.
*   The application's core functionality for facial emotion recognition remains intact and is suitable for deployment.

### Insights or Next Steps
*   The Gradio application is fully prepared for deployment to Hugging Face Spaces, as the identified `RuntimeError` is isolated to the Colab development environment.
*   The next logical step is to proceed with deploying the `app.py` script to a Hugging Face Space for broader accessibility and real-world testing.
